# Sentiment Analysis

Codice scalabile per calcolare il sentiment dei commenti.

Si implementa un sistema di **logging** per tracciare l'esecuzione del codice e facilitare il debugging.

In [2]:
# Importazioni
import pandas as pd
from multiprocessing import Pool
import logging
import yaml


In [3]:
# Configurazione del logging
logging.basicConfig(filename='../resources/preprocessing.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filemode='w')


In [4]:
# Funzione per caricare i dati
def load_data(file_path):
    logging.info(f'Caricamento dati da {file_path}')
    return pd.read_csv(file_path)

## Preprocessing

Esiste un tokenizzatore già addestrato su dati testuali quali tweet o commenti social. Può essere utilizzato anche per il italiano, anche se è stato originariamente progettato per testi in stile tweet in inglese. Tuttavia, poiché i tweet possono contenere caratteristiche multilingue simili (ad esempio emoticon, hashtag, abbreviazioni), TweetTokenizer può funzionare ragionevolmente bene per la tokenizzazione di testi in altre lingue, inclusa l'italiano.

In [5]:
from nltk.tokenize import TweetTokenizer
import string
import spacy
from nltk.corpus import stopwords
!python -m spacy download it_core_news_sm

     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
      --------------------------------------- 0.2/13.0 MB 3.5 MB/s eta 0:00:04
      --------------------------------------- 0.3/13.0 MB 2.9 MB/s eta 0:00:05
     - -------------------------------------- 0.5/13.0 MB 4.1 MB/s eta 0:00:04
     - -------------------------------------- 0.6/13.0 MB 3.5 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/13.0 MB 3.4 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/13.0 MB 3.3 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/13.0 MB 3.4 MB/s eta 0:00:04
     ---- ----------------------------------- 1.3/13.0 MB 3.5 MB/s eta 0:00:04
     ---- ----------------------------------- 1.5/13.0 MB 3.5 MB/s eta 0:00:04
     ----- ---------------------------------- 1.7/13.0 MB 3.6 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/13.0 MB 3.4 MB/s eta 0:00:04
     ----- ---------------------------------- 1.9/13.0 MB 3

2024-06-26 14:24:24.522129: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.



In [6]:
nlp = spacy.load("it_core_news_sm") # Carichiamo il modello italiano di spaCy

# Funzione per lemmatizzare una lista di token
def lemmatize_tokens(tokens):
    doc = nlp(" ".join(tokens))  # Converto i token in una stringa e processo con spaCy
    return [token.lemma_ for token in doc]  # Estraggo i lemmi

In [7]:
# Si utilizza una libreria specifica per la tokenizzazione multilingue come spaCy, 
# che ha modelli pre-addestrati per molte lingue, tra cui l'italiano.

tokening = TweetTokenizer() # Tokenizzazione per i tweet
stop = stopwords.words('italian')   # Carichiamo le stopwords italiane     
punctuation = string.punctuation    # Carichiamo la punteggiatura

# Tengo le parole non e più perché potrebbero essere importanti per il sentimento
stop = [item for item in stop if item not in ["non", "più"]]

# Funzione per eseguire il preprocessing dei dati
def preprocess_data(df):
    
    # Quando abbiamo a che fare con il testo dei social media, di solito vogliamo identificare URL, 
    # hashtag e emoticon come oggetti separati e non tokenizzarli in singoli caratteri:
    comments_tokenized = df.apply(TweetTokenizer().tokenize)

    # Rimuoviamo le stopword
    comments_tokenized_stop = comments_tokenized.apply(lambda x: [item for item in x if item not in stop])
    
    # Rimuoviamo la punteggiatura
    comments_tokenized_stop_punct = comments_tokenized_stop.apply(lambda x: [item for item in x if item not in punctuation])

    # Lemmatizzazione
    comments_tokenized_new_stem = comments_tokenized_stop_punct.apply(lemmatize_tokens)

    # Unisco i token lemmatizzati per formare le frasi preprocessate
    preprocessed_comments = comments_tokenized_new_stem.apply(lambda x: " ".join(x))

    return preprocessed_comments

## NRC Emotion Lexicon

L'**NRC Emotion Lexicon** ha annotazioni influenti per le parole inglesi. Nonostante alcune differenze culturali, è stato dimostrato che la maggior parte delle norme affettive sono stabili attraverso le lingue. Pertanto, forniamo versioni del lessico in oltre 100 lingue traducendo i termini inglesi utilizzando Google Translate (agosto 2022).

Il lessico è quindi disponibile per l'inglese ma anche per numerose altre lingue, tra cui l'italiano!

In [8]:
import pandas as pd

Emotion_Lexicon = "../resources/NRC-Emotion-Lexicon.xlsx"
lexicon_df = pd.read_excel(Emotion_Lexicon, engine="openpyxl")

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
lexicon = {}
for word, pos, neg, ant, ang, dis, fea, joy, sad, sur, tru in zip(lexicon_df["Italian Translation (Google Translate)"], lexicon_df["Positive"], lexicon_df["Negative"], lexicon_df["Anticipation"], lexicon_df["Anger"], lexicon_df["Disgust"], lexicon_df["Fear"], lexicon_df["Joy"], lexicon_df["Sadness"], lexicon_df["Surprise"], lexicon_df["Trust"]): 
    value = []
    if pos:
        value.append("positive")
    if neg:
        value.append("negative")
    if ant:
        value.append("anticipation")
    if ang:
        value.append("anger")
    if dis:
        value.append("disgust")
    if fea:
        value.append("fear")
    if joy:
        value.append("joy")
    if sad:
        value.append("sadness")
    if sur:
        value.append("surprise")
    if tru:
        value.append("trust")
    lexicon[str(word).lower()] = value #lower case

In [36]:
# Funzione che calcola il sentiment con NRC-Emotion-Lexicon
def nrc_score(df):
    # Inizializzo i contatori per i sentimenti
    positive_count = 0
    negative_count = 0
    
    # Inizializzo un dizionario per contare le emozioni
    emotion_count = {
        "anticipation": 0,
        "anger": 0,
        "disgust": 0,
        "fear": 0,
        "joy": 0,
        "sadness": 0,
        "surprise": 0,
        "trust": 0
    }
    
    # Analizzo ogni parola nella frase
    for word in df.split():  # La funzione split divide la stringa dagli spazi
        word = word.lower()
        if word in lexicon:
            values = lexicon[word]
            # Conto i sentimenti
            if "positive" in values:
                positive_count += 1
            if "negative" in values:
                negative_count += 1
            # Conta le emozioni
            for emotion in emotion_count.keys():
                if emotion in values:
                    emotion_count[emotion] += 1

    # Determino il sentimento prevalente
    if positive_count > negative_count:
        predominant_sentiment = "positive"
    elif negative_count > positive_count:
        predominant_sentiment = "negative"
    else:
        predominant_sentiment = "neutral"  

    # Determina l'emozione prevalente
    if all(value == 0 for value in emotion_count.values()):
        predominant_emotion = "none"  # Nessuna emozione prevalente
    else:
        predominant_emotion = max(emotion_count, key=emotion_count.get)
    
    # Restituisco una lista con i due valori prevalenti
    return [predominant_sentiment, predominant_emotion]

## Italian BERT Sentiment model

https://huggingface.co/neuraly/bert-base-italian-cased-sentiment

Questo modello esegue l'analisi del sentiment sulle frasi italiane. È stato addestrato a partire da un'istanza di bert-base-italian-cased, e messo a punto su un dataset italiano di tweets, raggiungendo su quest'ultimo l'82% di precisione.

In [12]:
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

In [13]:
# tokenizer
tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
# Load the model, use .cuda() to load it on the GPU
bert_model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
bert_model.eval()  # Imposta il modello in modalità valutazione

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32102, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [14]:
# Funzione che calcola il sentiment con Italian BERT Sentiment model
# Define a function to perform sentiment analysis using BERT
def bert_score(df):

    # Iterate through each row in the DataFrame
    for index, row in tqdm(df.iterrows()):
        sentence  = row.comment_body

        input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)

        # Create tensor, use .cuda() to transfer the tensor to GPU
        tensor = torch.tensor(input_ids).long()
        # Fake batch dimension
        tensor = tensor.unsqueeze(0)

        # Call the model and get the logits
        logits = bert_model(tensor)
        # Remove the fake batch dimension
        logits = logits.logits.squeeze(0)

        # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
        proba = nn.functional.softmax(logits, dim=0)

        # Unpack the tensor to obtain negative, neutral and positive probabilities
        negative, neutral, positive = proba

        df['sentiment_neg'] = proba[0].detach().numpy()
        df['sentiment_neut'] = proba[1].detach().numpy()
        df['sentiment_pos'] = proba[2].detach().numpy()

        df["BERT"] = df[['sentiment_neg', 'sentiment_pos', 'sentiment_neut']].idxmax(axis=1)
        # Salvo i risultati in un file
        df.to_csv('../results/BERT_results.csv', index=False)
        df.drop(['sentiment_neg', 'sentiment_neut', 'sentiment_pos'], axis=1, inplace=True)
        bert = df["BERT"].copy()
    return bert


## FEEL-IT: Emotion and Sentiment Classification for the Italian Language

https://huggingface.co/MilaNLProc/feel-it-italian-sentiment

Utilizzo un altro classificatore di sentiment, dotato di un corpus di riferimento di post Twitter italiani annotati con quattro emozioni fondamentali: rabbia, paura, gioia, tristezza. Comprimendoli, possiamo anche fare l’analisi del sentiment. 

PAPER: https://aclanthology.org/2021.wassa-1.8.pdf

In [15]:
from feel_it import SentimentClassifier
sentiment_classifier = SentimentClassifier()

In [16]:
from transformers import pipeline

classifier = pipeline("text-classification", model='MilaNLProc/feel-it-italian-sentiment', top_k=2)
sentiment_classifier.predict(["Oggi sono proprio contento!"])

['positive']

In [17]:
# Funzione per calcolare il sentiment con FEEL-IT
def feel_score(df):
    for i, row in df.iterrows():
        text = row['preprocessed_comment']  
        
        # Tokenizzo il testo e converto in tensori
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        
        # Eseguo la classificazione sul modello
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1).squeeze()
            score_positive = probabilities[1].item()
            score_negative = probabilities[0].item()
        
        # Assegno i punteggi al DataFrame
        df.at[i, 'sentiment_pos'] = score_positive  
        df.at[i, 'sentiment_neg'] = score_negative  
        
        # Determino l'etichetta di sentiment
        sentiment = 'positive' if score_positive >= score_negative else 'negative'
        df.at[i, 'FEEL'] = sentiment
        
        # Salvo i risultati in un file
        df.to_csv('../results/FEEL_IT_results.csv', index=False)

        df.drop(['sentiment_pos', 'sentiment_neg'], axis=1, inplace=True)

    return df
       
       

## Salvare i risultati

In [18]:
import os

# Funzione per salvare i risultati 
def save_partial_results(df, file_path):
    # Verifica se il file esiste
    if not os.path.isfile(file_path):
        df.to_csv(file_path, index=False, mode='w')
    else:
        df.to_csv(file_path, index=False, mode='a', header=False)
    logging.info(f'Salvati i risultati parziali in {file_path}')

## Predirre il sentiment

In [31]:
# Funzione per predire il sentiment usando un modello specifico
def predict_sentiment(model_name, df):
    # Implementa la logica di predizione per ciascun modello
    if(model_name == 'NRC'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df['NRC'] = df["preprocessed_comment"].apply(nrc_score)
        save_partial_results(df, '../results/NRC_results.csv')
    elif(model_name == 'BERT'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df['BERT'] = bert_score(df)
    elif(model_name == 'FEEL'):
        tqdm.pandas()  # Per visualizzare la progress bar con tqdm
        df['FEEL'] = feel_score(df)
    return df

In [20]:
# Funzione per eseguire il voto di maggioranza
def majority_vote(row):
    predictions = [row['BERT'], row['FEEL'], row['MULTILINGUAL']]
    if(row['BERT'] != row['FEEL'] != row['MULTILINGUAL']):
        return 'neutral'
    else:
        return max(set(predictions), key=predictions.count)

## Main

In [21]:
# Caricamento della configurazione
with open("../resources/config.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [22]:
 # Caricamento dei dati (Si utilizza il dataset generato al termine dell'analisi dei grafi, 
 # composto esclusivamente dai commenti che hanno contribuito attivamente ai dibattiti di ciascun post. 
 # Lo si vuole analizzare per capire se tali commenti abbiano un sentiment positivo o negativo.)
df = load_data(config['data']['input_path'])

In [34]:
# Creo una lista di id_post unici
id_posts = df['post_id'].unique()

print(id_posts)
print("Numero di post unici: ", len(id_posts))


# Creo una lista di id_post unici
debate_groups = df['debate_group'].unique()

['197vo6o' '19aeo2k' '1b6cg4q' '1cwqkqe' '1bulhj9' '1d5h5h6' '17z2hci'
 '17lese9' '10v8sey']
Numero di post unici:  9


In [37]:
# Filtro il dataset in modo tale da avere solo i commenti relativi ad un post

# Itero sugli id dei post
for id_post in id_posts:
    # Filtro il dataframe per l'id del post corrente
    df_filtered_post = df[df['post_id'] == id_post]
    for debate_group in df_filtered_post['debate_group'].unique():
        # Filtro il dataframe per il gruppo di dibattito corrente
        df_filtered_group = df_filtered_post[df_filtered_post['debate_group'] == debate_group]
        
        # Stampo il numero di commenti per il post corrente e gruppo di dibattito
        print(f"Numero di commenti per il post {id_post} che hanno partecipato al debate group {debate_group} : {len(df_filtered_group)}")
        
        # Isolo le uniche colonne che mi interessano
        df_fil = df_filtered_group[['comment_id', 'comment_body']].copy()
        
        # Estraggo solo i commenti
        comments = df_fil['comment_body']
        
        # Preprocesso i commenti
        preprocessed_comments = preprocess_data(comments)
        
        df_filtered_group["preprocessed_comment"] = preprocessed_comments
        
        # Utilizzo un indice booleano per assegnare direttamente i valori nel DataFrame originale
        df.loc[df_filtered_group.index, 'preprocessed_comment'] = preprocessed_comments
        
        # Salvo i risultati parziali
        save_partial_results(df.loc[df_filtered_group.index, ['comment_id', 'comment_body', 'preprocessed_comment']], config['data']['output_path'])

        
        # Predizioni con i modelli
        models = config['models']
        for mod in models:
            predict_sentiment(mod, df_filtered_group)
        
        # Salvo i risultati parziali
        save_partial_results(df_filtered_group, "../results/predictions.csv")

         


Numero di commenti per il post 197vo6o che hanno partecipato al debate group 1 : 27


C:\Users\franc\AppData\Local\Temp\ipykernel_15344\4143794906.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered_group["preprocessed_comment"] = preprocessed_comments
C:\Users\franc\AppData\Local\Temp\ipykernel_15344\4040951194.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NRC'] = df["preprocessed_comment"].apply(nrc_score)
0it [00:00, ?it/s]C:\Users\franc\AppData\Local\Temp\ipykernel_15344\3503236792.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

NameError: name 'model' is not defined

## Prove

In [ ]:
type(df_filtered_group.iloc[0])

pandas.core.series.Series

In [ ]:
nrc_score("Mi piace molto questo prodotto") 

['neutral', 'none']

In [ ]:
results_df = df[:10]

In [ ]:
results_df

,comment_id,post_id,debate_group,comment_user_name,commented_user_name,comment_body,comment_score,preprocessed_comment
0,ki3k2mh,197vo6o,1,SurprysE,Odd_Sentence_2618,BOIA SI! Basta guardare da dove sono registrat...,11,BOIA SI bastare guardare registrato account pr...
1,ki5i0gd,197vo6o,1,SurprysE,the_white_cloud,"Non fa una grinza, ma metti in movimento il tu...",1,non fare grinza metti movimento profilo renden...
2,ki4y2a5,197vo6o,1,SurprysE,the_white_cloud,Sono bot che fanno un minimo di attività per e...,1,essere bot minimo attività evitare essere bann...
3,ki3r883,197vo6o,1,theseawillclaim,zombilives,"Ma immagino, anche il suo brand è sempre in ro...",12,ma immagare Brand sempre rosso nonostante nego...
4,ki3jhkw,197vo6o,2,Polaroid1793,TURBINEFABRIK74,Non ho mai visto una persona in vita mia camb...,3,NaN
5,ki3rgah,197vo6o,1,zombilives,theseawillclaim,"ma infatti è quello che dico io, è allucinante...",6,infatti dire allucinante pollo gente guadagnar...
6,ki3pxf1,197vo6o,1,zombilives,theseawillclaim,infatti se ricordo bene la collaborazione dell...,8,infatti ricordare bene collaborazione signore ...
7,ki3pi87,197vo6o,1,zombilives,David0ne86,vero o LMAO,2,vero LMAO
8,ki3pfur,197vo6o,1,zombilives,SuperCrazyAlbatross,ma tanto tutte le cose stile charity non vanno...,1,tanto tutto cosa stile charity non andare mai ...
9,ki3porw,197vo6o,1,zombilives,Lassemb,e si lei e il marito fanno i comunisti col ric...,7,marito comunista richard mille


In [ ]:
dg = df_filtered.copy()

In [ ]:
dg

,comment_id,post_id,debate_group,comment_user_name,commented_user_name,comment_body,comment_score,preprocessed_comment
1368,j7gqokb,10v8sey,34,sbranzo,Pandinuuu24,Le uova sono le mestruazioni delle galline,26,il uovo mestruazione gallino
1486,j7gqsij,10v8sey,34,Pandinuuu24,sbranzo,Siamo pure feticisti,22,essere pure feticirre
1487,j7gq4s9,10v8sey,34,Pandinuuu24,sbranzo,Un secondo…i comunisti mangiavano bambini…noi ...,33,uno secondo … comunista mangiare bambino … man...
1488,j8craw0,10v8sey,34,Pandinuuu24,Daubeny_il_glorioso,Stalin risorge per stringerti la mano e Marx r...,3,Stalin risorge stringergere mano Marx riscrivo...
1505,j7jmq41,10v8sey,34,Daubeny_il_glorioso,Pandinuuu24,Io mangio anche i bambini oltre alle uova... Q...,6,io mangiare bambino oltre uovo ... questo fare...
1506,j7hv066,10v8sey,34,ingframin,Pandinuuu24,Ben detto compagno,1,Ben dire compagno
1507,j7jxlao,10v8sey,34,ResponsibilityOk3804,Pandinuuu24,r/unexpectedcomunism,1,r unexpectedcomunism


In [ ]:
# Predizioni con i modelli
models = config['models']
for mod in models:
    predict_sentiment(mod, dg)

TypeError: 'DataFrame' object is not callable

In [ ]:
dg

,comment_id,post_id,debate_group,comment_user_name,commented_user_name,comment_body,comment_score,preprocessed_comment,NRC,BERT
1368,j7gqokb,10v8sey,34,sbranzo,Pandinuuu24,Le uova sono le mestruazioni delle galline,26,il uovo mestruazione gallino,"[neutral, none]",sentiment_neut
1486,j7gqsij,10v8sey,34,Pandinuuu24,sbranzo,Siamo pure feticisti,22,essere pure feticirre,"[neutral, none]",sentiment_neut
1487,j7gq4s9,10v8sey,34,Pandinuuu24,sbranzo,Un secondo…i comunisti mangiavano bambini…noi ...,33,uno secondo … comunista mangiare bambino … man...,"[neutral, anticipation]",sentiment_neut
1488,j8craw0,10v8sey,34,Pandinuuu24,Daubeny_il_glorioso,Stalin risorge per stringerti la mano e Marx r...,3,Stalin risorge stringergere mano Marx riscrivo...,"[negative, none]",sentiment_neut
1505,j7jmq41,10v8sey,34,Daubeny_il_glorioso,Pandinuuu24,Io mangio anche i bambini oltre alle uova... Q...,6,io mangiare bambino oltre uovo ... questo fare...,"[neutral, anticipation]",sentiment_neut
1506,j7hv066,10v8sey,34,ingframin,Pandinuuu24,Ben detto compagno,1,Ben dire compagno,"[positive, none]",sentiment_neut
1507,j7jxlao,10v8sey,34,ResponsibilityOk3804,Pandinuuu24,r/unexpectedcomunism,1,r unexpectedcomunism,"[neutral, none]",sentiment_neut


In [ ]:
lovah = dg['comment_body'].apply(bert_score)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
